In [1]:
import sys
sys.path.append("..")
import matplotlib.pyplot as plt
from scziDesk_preprocess import *
from scziDesk_network import *
from scziDesk_utils import *
import argparse
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from collections import Counter
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import h5py
import time
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, silhouette_score, calinski_harabasz_score
import umap
import glob2
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

/opt/conda/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,
Using TensorFlow backend.


In [2]:
for category in [  "balanced_data", "imbalanced_data",#"real_data", #
                ]:

    path= ".."
    if category in ["balanced_data", "imbalanced_data"]:
        files = glob2.glob(f'{path}/R/simulated_data/{category}/*.h5')
        files = [f[len(f"{path}/R/simulated_data/{category}/"):-3] for f in files]
    else:
        files = glob2.glob(f'{path}/real_data/*.h5')
        files = [f[len(f"{path}/real_data/"):-3] for f in files]
    print(files)

    df = pd.DataFrame(columns = ["dataset", "ARI", "NMI", "sil", "run", "time", "pred", "cal"])
    for dataset in files:
        if category in ["balanced_data", "imbalanced_data"]:
            data_mat = h5py.File(f"{path}/R/simulated_data/{category}/{dataset}.h5","r")
        else:
            data_mat = h5py.File(f"{path}/real_data/{dataset}.h5","r")

        Y = np.array(data_mat['Y'])
        X = np.array(data_mat['X'])
        print(f">>>>dataset {dataset}")
        if X.shape[0] > 10000:
            continue
        n_clusters =  np.unique(Y).shape[0]

        X = np.ceil(X).astype(np.int)
        orig_X = X.copy()
        adata = sc.AnnData(X)
        adata.obs['Group'] = Y

        adata = sc.AnnData(X)
        adata.obs['Group'] = Y
        adata = normalize(adata,
                          copy=True,
                          highly_genes=500,
                          size_factors=True,
                          normalize_input=True,
                          logtrans_input=True)
        X = adata.X.astype(np.float32)
        Y = np.array(adata.obs["Group"])

        for run in range(3):
            start = time.time()
            seed = run
            np.random.seed(seed)
#             pca_data = PCA(50, random_state = seed).fit_transform(X)
            pca_data = umap.UMAP(n_components = 20).fit_transform(X)
            kmeans = KMeans(n_clusters=n_clusters, init="k-means++", random_state=seed)
            pred = kmeans.fit_predict(pca_data)

            elapsed = time.time() - start
            ARI = adjusted_rand_score(Y, pred)
            NMI = np.around(normalized_mutual_info_score(Y, pred), 5)
            ss = silhouette_score(pca_data,pred)
            cal = calinski_harabasz_score(pca_data, pred)

            df.loc[df.shape[0]] = [
                dataset, ARI, NMI, ss, run, elapsed, pred, cal
            ]
            df.to_pickle(f"../output/pickle_results/{category}/{category}_umap20_kmeans_scaling.pkl")

['data_1c8', 'data_-1c4', 'data_-1c8', 'data_0c4', 'data_0c8', 'data_0c16', 'data_1.5c4', 'data_1c4', 'data_1.5c8', 'data_1.5c16', 'data_-1c16', 'data_1c16']
>>>>dataset data_1c8
>>>>dataset data_-1c4
>>>>dataset data_-1c8
>>>>dataset data_0c4
>>>>dataset data_0c8
>>>>dataset data_0c16
>>>>dataset data_1.5c4
>>>>dataset data_1c4
>>>>dataset data_1.5c8
>>>>dataset data_1.5c16
>>>>dataset data_-1c16
>>>>dataset data_1c16
['data_1c8', 'data_-1c4', 'data_-1c8', 'data_0c4', 'data_0c8', 'data_0c16', 'data_1.5c4', 'data_1c4', 'data_1.5c8', 'data_1.5c16', 'data_-1c16', 'data_1c16']
>>>>dataset data_1c8
>>>>dataset data_-1c4
>>>>dataset data_-1c8
>>>>dataset data_0c4
>>>>dataset data_0c8
>>>>dataset data_0c16
>>>>dataset data_1.5c4
>>>>dataset data_1c4
>>>>dataset data_1.5c8
>>>>dataset data_1.5c16
>>>>dataset data_-1c16
>>>>dataset data_1c16


In [3]:
df.mean()

ARI        0.551928
NMI        0.626065
sil        0.496767
run        1.000000
time      10.403718
cal     9780.853071
dtype: float64

In [4]:
df.groupby("dataset").mean()

,ARI,NMI,sil,time,cal
dataset,,,,,
10X_PBMC,0.587730,0.716707,0.370691,0.245260,712.954770
Adam,0.518795,0.690063,0.193993,0.248612,315.108018
Muraro,0.697615,0.745110,0.263742,0.143753,527.967632
Quake_10x_Bladder,0.428698,0.571783,0.619011,0.099293,441.627325
Quake_10x_Limb_Muscle,0.832756,0.862610,0.156571,0.197953,346.149685
Quake_10x_Spleen,0.858964,0.761780,0.468265,0.271574,1001.364294
Quake_Smart-seq2_Diaphragm,0.766001,0.822713,0.311358,0.067705,122.458027
Quake_Smart-seq2_Limb_Muscle,0.634776,0.785507,0.372249,0.078961,131.776675
Quake_Smart-seq2_Lung,0.577494,0.749997,0.310795,0.156092,166.845150
